In [1]:
from cStringIO import StringIO
import fastavro

In [2]:
import pyspark

In [3]:
sc = pyspark.SparkContext()

In [12]:
pickled_page_views = sc.pickleFile(
    "./SuperWebAnalytics/master/page_view/")

In [13]:
pickled_page_views.count()

7254929

In [15]:
pickled_page_views.take(2)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 53, localhost): java.net.SocketException: Connection reset by peer: socket write error
	at java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.net.SocketOutputStream.socketWrite(SocketOutputStream.java:109)
	at java.net.SocketOutputStream.write(SocketOutputStream.java:153)
	at java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:82)
	at java.io.BufferedOutputStream.write(BufferedOutputStream.java:126)
	at java.io.DataOutputStream.write(DataOutputStream.java:107)
	at java.io.FilterOutputStream.write(FilterOutputStream.java:97)
	at org.apache.spark.api.python.PythonRDD$.org$apache$spark$api$python$PythonRDD$$write$1(PythonRDD.scala:413)
	at org.apache.spark.api.python.PythonRDD$$anonfun$writeIteratorToSt

In [17]:
raw_page_views = sc.binaryFiles("./SuperWebAnalytics/master/page_view/*")

In [18]:
avro_page_views = raw_page_views.map(lambda t: StringIO(t[1]))
json_page_views = avro_page_views.flatMap(fastavro.reader)

In [19]:
page_views = raw_page_views.flatMap(lambda x: x[1].split("\n")).filter(
    len)#.values().map(eval)

In [20]:
from requests.utils import urlparse
def normalize_url(datum):
    pr = urlparse(datum['dataunit']['page_view']['page']['url'])
    datum['dataunit']['page_view']['page']['url'] = pr.hostname + pr.path
    return datum

In [21]:
normalized_page_views = pickled_page_views.map(normalize_url)

In [22]:
normalized_page_views.first()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 54, localhost): java.net.SocketException: Connection reset by peer: socket write error
	at java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.net.SocketOutputStream.socketWrite(SocketOutputStream.java:109)
	at java.net.SocketOutputStream.write(SocketOutputStream.java:153)
	at java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:82)
	at java.io.BufferedOutputStream.write(BufferedOutputStream.java:126)
	at java.io.DataOutputStream.write(DataOutputStream.java:107)
	at java.io.FilterOutputStream.write(FilterOutputStream.java:97)
	at org.apache.spark.api.python.PythonRDD$.org$apache$spark$api$python$PythonRDD$$write$1(PythonRDD.scala:413)
	at org.apache.spark.api.python.PythonRDD$$anonfun$writeIteratorToSt

Deduplicate pageviews
---------------------

In [11]:
import simplejson as json
distinct_normalized_page_views = normalized_page_views.map(json.dumps).distinct().map(json.loads)

In [12]:
distinct_normalized_page_views.first()

{'dataunit': {'page_view': {'nonce': 4028254285,
   'page': {'url': 'mysite.com/blog'},
   'person': {'cookie': 'PQRST'}}},
 'pedigree': {'true_as_of_secs': 1438913491}}

Pageviews over time
---------------------

In [13]:
granularity = {'h': 60 * 60}
granularity['d'] = granularity['h'] * 24
granularity['w'] = granularity['d'] * 7
granularity['m'] = granularity['w'] * 4
granularity

{'d': 86400, 'h': 3600, 'm': 2419200, 'w': 604800}

In [54]:
int(1438913491 / (60*60)) * (60*60)

1438912800

In [55]:
1438913491 - 1438913491 % (60*60)

1438912800

In [14]:
def hourly(datum):
    url = datum['dataunit']['page_view']['page']['url']
    true_as_of_secs = datum['pedigree']['true_as_of_secs']
    return (url, 'h', true_as_of_secs - true_as_of_secs % granularity['h']), datum

In [15]:
hourly_rollup = distinct_normalized_page_views.map(hourly).countByKey()

In [16]:
i, c = zip(*hourly_rollup.items())

In [17]:
import pandas as pd

In [19]:
pd.DataFrame({'count': c}, index=pd.MultiIndex.from_tuples(i, names=('url', 'hour'))).reset_index()

,url,hour,count
0,mysite.com/,1438912800,300054
1,mysite.com/blog,1438624800,101732
2,mysite.com/blog,1438912800,899778
3,mysite.com/,1438624800,101212
